# Pyspark SQL

Este kernel possui alguns exemplos de código do módulo pyspark.sql.
Apresenta exemplos de funções que implementam instruções SQL, como: SELECT, WHERE, GROUP BY, ORDER BY, PARTITION BY, entre outras.


São usados dois datasets para os exemplos:
*   housing.csv: contém informações sobre imóveis no estado da Califórnia;
*   wc2018-players.csv: contém informações sobre jogadores de futebol da Copa do Mundo 2018.

In [1]:
%pip install ipython-autotime
%pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=4781699e4055a144ad0945cf48bc04af1eb302573980f6191efdfd12d6dc65d3
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


Trecho de código opcional que resolve problema de compatibilidade entre a linguagem Python e o PySpark.

In [2]:
%%script echo 'ignore cell'
import os
import sys

# resolver problema de versão python x pyspark workers
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

ignore cell


# Imports básicos.

In [3]:
from google.colab          import drive, files
from pyspark.sql           import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types     import *
from pyspark.sql.window    import Window

%load_ext autotime

time: 436 µs (started: 2023-09-19 16:48:42 +00:00)


# Início da sessão.

In [4]:
drive.mount('/content/drive', force_remount=True)
spark = SparkSession.builder.master('local').appName('pyspark_app').getOrCreate()
spark

Mounted at /content/drive


time: 1min 18s (started: 2023-09-19 16:48:42 +00:00)


In [5]:
df = spark.read.csv("/content/drive/MyDrive/datasets/housing/housing.csv", header=True, inferSchema=True, encoding='utf-8')
print(type(df))
print(f'rows: {df.count()}')
print(f'cols: {len(df.columns)}')

<class 'pyspark.sql.dataframe.DataFrame'>
rows: 20640
cols: 10
time: 15.9 s (started: 2023-09-19 16:50:00 +00:00)


# Funções descritivas básicas.

In [6]:
df.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  -122.25|   37.85|              

In [7]:
print(df.columns)

['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value', 'ocean_proximity']
time: 870 µs (started: 2023-09-19 16:50:17 +00:00)


In [8]:
df.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)

time: 16.7 ms (started: 2023-09-19 16:50:17 +00:00)


In [9]:
df.describe().show()

+-------+-------------------+-----------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+---------------+
|summary|          longitude|         latitude|housing_median_age|       total_rooms|    total_bedrooms|        population|       households|     median_income|median_house_value|ocean_proximity|
+-------+-------------------+-----------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+---------------+
|  count|              20640|            20640|             20640|             20640|             20433|             20640|            20640|             20640|             20640|          20640|
|   mean|-119.56970445736148| 35.6318614341087|28.639486434108527|2635.7630813953488| 537.8705525375618|1425.4767441860465|499.5396802325581|3.8706710029070246|206855.81690891474|           null|
| stddev|  2.0035317

# Funções úteis para feature engineering.

## Atributos

### Renomeação
Transformando tudo em letras maiúsculas.

In [10]:
upper = [column.upper() for column in df.columns]
for column, up in zip(df.columns, upper):
  df = df.withColumnRenamed(column, up)
print(df.columns)

['LONGITUDE', 'LATITUDE', 'HOUSING_MEDIAN_AGE', 'TOTAL_ROOMS', 'TOTAL_BEDROOMS', 'POPULATION', 'HOUSEHOLDS', 'MEDIAN_INCOME', 'MEDIAN_HOUSE_VALUE', 'OCEAN_PROXIMITY']
time: 102 ms (started: 2023-09-19 16:50:23 +00:00)


Transformando tudo em letras minúsculas.

In [11]:
lower = [column.lower() for column in df.columns]
for column, low in zip(df.columns, lower):
  df = df.withColumnRenamed(column, low)
print(df.columns)

['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value', 'ocean_proximity']
time: 142 ms (started: 2023-09-19 16:50:23 +00:00)


Atribuindo um 'alias' a cada atributo selecionado. Só pode ser feito através da função col() que retorna um objeto Column.


In [12]:
lat = col('latitude').alias('lat')
lon = col('longitude').alias('lon')

print(lat)
df.select([lat, lon]).show(5)

Column<'latitude AS lat'>
+-----+-------+
|  lat|    lon|
+-----+-------+
|37.88|-122.23|
|37.86|-122.22|
|37.85|-122.24|
|37.85|-122.25|
|37.85|-122.25|
+-----+-------+
only showing top 5 rows

time: 300 ms (started: 2023-09-19 16:50:24 +00:00)


### Instrução: SELECT

In [13]:
print(type(df.select(['longitude', 'latitude', 'households'])))
df.select(['longitude', 'latitude', 'households']).show(5)

<class 'pyspark.sql.dataframe.DataFrame'>
+---------+--------+----------+
|longitude|latitude|households|
+---------+--------+----------+
|  -122.23|   37.88|     126.0|
|  -122.22|   37.86|    1138.0|
|  -122.24|   37.85|     177.0|
|  -122.25|   37.85|     219.0|
|  -122.25|   37.85|     259.0|
+---------+--------+----------+
only showing top 5 rows

time: 290 ms (started: 2023-09-19 16:50:24 +00:00)


Forma alternativa utilizando a função col() que retorna um objeto da classe Column.

In [14]:
print(type(df.select([col('latitude'), col('longitude'), col('households')])))
df.select([col('latitude'), col('longitude'), col('households')]).show(5)

<class 'pyspark.sql.dataframe.DataFrame'>
+--------+---------+----------+
|latitude|longitude|households|
+--------+---------+----------+
|   37.88|  -122.23|     126.0|
|   37.86|  -122.22|    1138.0|
|   37.85|  -122.24|     177.0|
|   37.85|  -122.25|     219.0|
|   37.85|  -122.25|     259.0|
+--------+---------+----------+
only showing top 5 rows

time: 267 ms (started: 2023-09-19 16:50:24 +00:00)


### Criação
Atribuo um valor literal(True) à nova coluna chamada 'new_col'. A função lit() retorna um objeto Column.

In [15]:
df.withColumn('new_col', lit(True)).show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|new_col|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|   true|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|   true|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|   true|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|   

Nova coluna criada a partir de uma operação matemática entre outras duas. Neste caso é preciso que os atributos sejam numéricos.

In [16]:
result = df['total_bedrooms'] / df['total_rooms']
print(type(result))
df.withColumn('new_col', result).show(5)

<class 'pyspark.sql.column.Column'>
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|            new_col|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|0.14659090909090908|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|0.15579659106916466|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|0.12951601908657123|
|  -122.25|   37

Nova coluna usando a função substring().

In [17]:
df.withColumn('new_col', substring('ocean_proximity', 1, 4)).show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|new_col|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|   NEAR|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|   NEAR|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|   NEAR|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|   

Concatenando dois atributos para formar um novo.

In [18]:
df.withColumn('new_col', concat(df['latitude'], df['longitude'])).show(10)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|     new_col|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|37.88-122.23|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|37.86-122.22|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|37.85-122.24|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|      

In [19]:
df.withColumn('new_col', concat_ws(' # ', df['latitude'], df['longitude'])).show(10)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|        new_col|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|37.88 # -122.23|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|37.86 # -122.22|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|37.85 # -122.24|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0

### Instrução: DROP

In [20]:
df = df.drop('population')
df.show(5)

+---------+--------+------------------+-----------+--------------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1627.0|         280.0|     259.0|       3.8462|          342200.0| 

## Valores NaN

### Identificação

In [21]:
for column in df.columns:
  mask = df['longitude'].isNull()
  nan_amount = df.filter(mask).count()
  print(f'{column}: {nan_amount}')

longitude: 0
latitude: 0
housing_median_age: 0
total_rooms: 0
total_bedrooms: 0
households: 0
median_income: 0
median_house_value: 0
ocean_proximity: 0
time: 3.44 s (started: 2023-09-19 16:50:26 +00:00)


--------------

# Outro dataset
Vou usar outro dataset que contenha datas em formato string para fazer a conversão de string para o formato DateType.

In [22]:
df = spark.read.csv("/content/drive/MyDrive/datasets/wc2018-players.csv", header=True, inferSchema=True, encoding='utf-8')
print(type(df))
print(f'rows: {df.count()}')
print(f'cols: {len(df.columns)}')
df.show(5)

<class 'pyspark.sql.dataframe.DataFrame'>
rows: 736
cols: 9
+---------+---+----+------------------+----------+----------+--------------------+------+------+
|     Team|  #|Pos.| FIFA Popular Name|Birth Date|Shirt Name|                Club|Height|Weight|
+---------+---+----+------------------+----------+----------+--------------------+------+------+
|Argentina|  3|  DF|TAGLIAFICO Nicolas|31.08.1992|TAGLIAFICO|      AFC Ajax (NED)|   169|    65|
|Argentina| 22|  MF|    PAVON Cristian|21.01.1996|     PAVÓN|CA Boca Juniors (...|   169|    65|
|Argentina| 15|  MF|    LANZINI Manuel|15.02.1993|   LANZINI|West Ham United F...|   167|    66|
|Argentina| 18|  DF|    SALVIO Eduardo|13.07.1990|    SALVIO|    SL Benfica (POR)|   167|    69|
|Argentina| 10|  FW|      MESSI Lionel|24.06.1987|     MESSI|  FC Barcelona (ESP)|   170|    72|
+---------+---+----+------------------+----------+----------+--------------------+------+------+
only showing top 5 rows

time: 1.28 s (started: 2023-09-19 16:50:30

In [23]:
df.printSchema()

root
 |-- Team: string (nullable = true)
 |-- #: integer (nullable = true)
 |-- Pos.: string (nullable = true)
 |-- FIFA Popular Name: string (nullable = true)
 |-- Birth Date: string (nullable = true)
 |-- Shirt Name: string (nullable = true)
 |-- Club: string (nullable = true)
 |-- Height: integer (nullable = true)
 |-- Weight: integer (nullable = true)

time: 2.59 ms (started: 2023-09-19 16:50:31 +00:00)


Vou renomear as colunas convertendo-as para letras minúsculas.

In [24]:
lower = [column.lower() for column in df.columns]
for column, low in zip(df.columns, lower):
  df = df.withColumnRenamed(column, low)

df = df.withColumnRenamed('pos.', 'pos')
print(df.columns)

['team', '#', 'pos', 'fifa popular name', 'birth date', 'shirt name', 'club', 'height', 'weight']
time: 67.5 ms (started: 2023-09-19 16:50:31 +00:00)


Note que 'birth date' está no formato string e também o separador dos componentes YYY, MM, DD é um ponto. Abaixo vemos uma forma de extrair os componentes.

In [25]:
dia = udf(lambda date:date.split('.')[0])
mes = udf(lambda date:date.split('.')[1])
ano = udf(lambda date:date.split('.')[2])

df = df.withColumn('dia', dia('birth date'))
df = df.withColumn('mes', mes('birth date'))
df = df.withColumn('ano', ano('birth date'))

df.show(5)

+---------+---+---+------------------+----------+----------+--------------------+------+------+---+---+----+
|     team|  #|pos| fifa popular name|birth date|shirt name|                club|height|weight|dia|mes| ano|
+---------+---+---+------------------+----------+----------+--------------------+------+------+---+---+----+
|Argentina|  3| DF|TAGLIAFICO Nicolas|31.08.1992|TAGLIAFICO|      AFC Ajax (NED)|   169|    65| 31| 08|1992|
|Argentina| 22| MF|    PAVON Cristian|21.01.1996|     PAVÓN|CA Boca Juniors (...|   169|    65| 21| 01|1996|
|Argentina| 15| MF|    LANZINI Manuel|15.02.1993|   LANZINI|West Ham United F...|   167|    66| 15| 02|1993|
|Argentina| 18| DF|    SALVIO Eduardo|13.07.1990|    SALVIO|    SL Benfica (POR)|   167|    69| 13| 07|1990|
|Argentina| 10| FW|      MESSI Lionel|24.06.1987|     MESSI|  FC Barcelona (ESP)|   170|    72| 24| 06|1987|
+---------+---+---+------------------+----------+----------+--------------------+------+------+---+---+----+
only showing top 5 

A conversão de "birth date" para o formato DateType pode ser feita da seguinte forma.

In [26]:
df.withColumn('data nascimento', to_date(col("birth date"), "dd.MM.yyyy")).printSchema()
# ou
#df.withColumn('Data', to_date(col("birth date"), "dd.MM.yyyy").cast(DateType())).printSchema()

root
 |-- team: string (nullable = true)
 |-- #: integer (nullable = true)
 |-- pos: string (nullable = true)
 |-- fifa popular name: string (nullable = true)
 |-- birth date: string (nullable = true)
 |-- shirt name: string (nullable = true)
 |-- club: string (nullable = true)
 |-- height: integer (nullable = true)
 |-- weight: integer (nullable = true)
 |-- dia: string (nullable = true)
 |-- mes: string (nullable = true)
 |-- ano: string (nullable = true)
 |-- data nascimento: date (nullable = true)

time: 96.2 ms (started: 2023-09-19 16:50:33 +00:00)


Outra conversão de tipo com a função cast().

In [27]:
df.withColumn('height', col('height').cast(FloatType())).show(5)

+---------+---+---+------------------+----------+----------+--------------------+------+------+---+---+----+
|     team|  #|pos| fifa popular name|birth date|shirt name|                club|height|weight|dia|mes| ano|
+---------+---+---+------------------+----------+----------+--------------------+------+------+---+---+----+
|Argentina|  3| DF|TAGLIAFICO Nicolas|31.08.1992|TAGLIAFICO|      AFC Ajax (NED)| 169.0|    65| 31| 08|1992|
|Argentina| 22| MF|    PAVON Cristian|21.01.1996|     PAVÓN|CA Boca Juniors (...| 169.0|    65| 21| 01|1996|
|Argentina| 15| MF|    LANZINI Manuel|15.02.1993|   LANZINI|West Ham United F...| 167.0|    66| 15| 02|1993|
|Argentina| 18| DF|    SALVIO Eduardo|13.07.1990|    SALVIO|    SL Benfica (POR)| 167.0|    69| 13| 07|1990|
|Argentina| 10| FW|      MESSI Lionel|24.06.1987|     MESSI|  FC Barcelona (ESP)| 170.0|    72| 24| 06|1987|
+---------+---+---+------------------+----------+----------+--------------------+------+------+---+---+----+
only showing top 5 

Remoção de uma coluna.

In [28]:
df = df.withColumn('new_col', lit(True))
df = df.drop('new_col')
df.show(5)

+---------+---+---+------------------+----------+----------+--------------------+------+------+---+---+----+
|     team|  #|pos| fifa popular name|birth date|shirt name|                club|height|weight|dia|mes| ano|
+---------+---+---+------------------+----------+----------+--------------------+------+------+---+---+----+
|Argentina|  3| DF|TAGLIAFICO Nicolas|31.08.1992|TAGLIAFICO|      AFC Ajax (NED)|   169|    65| 31| 08|1992|
|Argentina| 22| MF|    PAVON Cristian|21.01.1996|     PAVÓN|CA Boca Juniors (...|   169|    65| 21| 01|1996|
|Argentina| 15| MF|    LANZINI Manuel|15.02.1993|   LANZINI|West Ham United F...|   167|    66| 15| 02|1993|
|Argentina| 18| DF|    SALVIO Eduardo|13.07.1990|    SALVIO|    SL Benfica (POR)|   167|    69| 13| 07|1990|
|Argentina| 10| FW|      MESSI Lionel|24.06.1987|     MESSI|  FC Barcelona (ESP)|   170|    72| 24| 06|1987|
+---------+---+---+------------------+----------+----------+--------------------+------+------+---+---+----+
only showing top 5 

Criando backup do dataframe

In [29]:
df_backup = df

time: 2.62 ms (started: 2023-09-19 16:50:35 +00:00)


# Instrução: WHERE
Função que implementa a instrução WHERE do SQL. A string argumento da função where() deve seguir o padrão SQL.

Uma alternativa é a função filter() que pode ser usada da mesma forma.

In [30]:
df.where("team = 'Brazil'").show(5)

+------+---+---+-----------------+----------+-----------+--------------------+------+------+---+---+----+
|  team|  #|pos|fifa popular name|birth date| shirt name|                club|height|weight|dia|mes| ano|
+------+---+---+-----------------+----------+-----------+--------------------+------+------+---+---+----+
|Brazil| 18| MF|             FRED|05.03.1993|       FRED|FC Shakhtar Donet...|   169|    64| 05| 03|1993|
|Brazil| 21| FW|           TAISON|13.01.1988|     TAISON|FC Shakhtar Donet...|   172|    64| 13| 01|1988|
|Brazil| 17| MF|      FERNANDINHO|04.05.1985|FERNANDINHO|Manchester City F...|   179|    67| 04| 05|1985|
|Brazil| 22| DF|           FAGNER|11.06.1989|     FAGNER|SC Corinthians (BRA)|   168|    67| 11| 06|1989|
|Brazil| 10| FW|           NEYMAR|05.02.1992|  NEYMAR JR|Paris Saint-Germa...|   175|    68| 05| 02|1992|
+------+---+---+-----------------+----------+-----------+--------------------+------+------+---+---+----+
only showing top 5 rows

time: 946 ms (started

In [31]:
mask = df['team'] == 'Argentina'
print(mask)
df.where(mask).show(5)

Column<'(team = Argentina)'>
+---------+---+---+------------------+----------+----------+--------------------+------+------+---+---+----+
|     team|  #|pos| fifa popular name|birth date|shirt name|                club|height|weight|dia|mes| ano|
+---------+---+---+------------------+----------+----------+--------------------+------+------+---+---+----+
|Argentina|  3| DF|TAGLIAFICO Nicolas|31.08.1992|TAGLIAFICO|      AFC Ajax (NED)|   169|    65| 31| 08|1992|
|Argentina| 22| MF|    PAVON Cristian|21.01.1996|     PAVÓN|CA Boca Juniors (...|   169|    65| 21| 01|1996|
|Argentina| 15| MF|    LANZINI Manuel|15.02.1993|   LANZINI|West Ham United F...|   167|    66| 15| 02|1993|
|Argentina| 18| DF|    SALVIO Eduardo|13.07.1990|    SALVIO|    SL Benfica (POR)|   167|    69| 13| 07|1990|
|Argentina| 10| FW|      MESSI Lionel|24.06.1987|     MESSI|  FC Barcelona (ESP)|   170|    72| 24| 06|1987|
+---------+---+---+------------------+----------+----------+--------------------+------+------+---+

In [32]:
mask = (col('shirt name') == 'MESSI')
print(mask)
df.filter(mask).show(5)

Column<'(shirt name = MESSI)'>
+---------+---+---+-----------------+----------+----------+------------------+------+------+---+---+----+
|     team|  #|pos|fifa popular name|birth date|shirt name|              club|height|weight|dia|mes| ano|
+---------+---+---+-----------------+----------+----------+------------------+------+------+---+---+----+
|Argentina| 10| FW|     MESSI Lionel|24.06.1987|     MESSI|FC Barcelona (ESP)|   170|    72| 24| 06|1987|
+---------+---+---+-----------------+----------+----------+------------------+------+------+---+---+----+

time: 1.03 s (started: 2023-09-19 16:50:37 +00:00)


## Filtros compostos

In [33]:
mask = ("team = 'Brazil' AND height < 170")
print(mask)
df.where(mask).show(5)

team = 'Brazil' AND height < 170
+------+---+---+-----------------+----------+----------+--------------------+------+------+---+---+----+
|  team|  #|pos|fifa popular name|birth date|shirt name|                club|height|weight|dia|mes| ano|
+------+---+---+-----------------+----------+----------+--------------------+------+------+---+---+----+
|Brazil| 18| MF|             FRED|05.03.1993|      FRED|FC Shakhtar Donet...|   169|    64| 05| 03|1993|
|Brazil| 22| DF|           FAGNER|11.06.1989|    FAGNER|SC Corinthians (BRA)|   168|    67| 11| 06|1989|
+------+---+---+-----------------+----------+----------+--------------------+------+------+---+---+----+

time: 861 ms (started: 2023-09-19 16:50:38 +00:00)


In [34]:
mask = (col('team') == 'Brazil') & (col('height') < 170)
print(mask)
df.where(mask).show(5)

Column<'((team = Brazil) AND (height < 170))'>
+------+---+---+-----------------+----------+----------+--------------------+------+------+---+---+----+
|  team|  #|pos|fifa popular name|birth date|shirt name|                club|height|weight|dia|mes| ano|
+------+---+---+-----------------+----------+----------+--------------------+------+------+---+---+----+
|Brazil| 18| MF|             FRED|05.03.1993|      FRED|FC Shakhtar Donet...|   169|    64| 05| 03|1993|
|Brazil| 22| DF|           FAGNER|11.06.1989|    FAGNER|SC Corinthians (BRA)|   168|    67| 11| 06|1989|
+------+---+---+-----------------+----------+----------+--------------------+------+------+---+---+----+

time: 545 ms (started: 2023-09-19 16:50:39 +00:00)


# Instrução: GROUP BY

Tendo uma coluna como referência, todas as linhas onde os valores dessa coluna são iguais são "colapsadas" em apenas uma. É preciso especificar o que deve ser feito com as outras colunas caso contrário elas serão ignoradas. Normalmente aplicamos funções de estatística descritiva.

In [35]:
df.groupBy('team').mean('weight').orderBy('avg(weight)', ascending=True).show(10)

+--------------+-----------------+
|          team|      avg(weight)|
+--------------+-----------------+
|         Japan|71.52173913043478|
|  Saudi Arabia|73.04347826086956|
|      Portugal| 73.6086956521739|
|        Mexico|74.08695652173913|
|    Costa Rica| 74.1304347826087|
|Korea Republic|74.43478260869566|
|       Uruguay| 74.6086956521739|
|       Morocco|74.65217391304348|
|         Spain|74.73913043478261|
|       Tunisia|             75.0|
+--------------+-----------------+
only showing top 10 rows

time: 792 ms (started: 2023-09-19 16:50:39 +00:00)


Para especificar qual função de agregação deve ser usada em cada coluna podemos usar a função agg().

In [36]:
df.groupBy('team').agg({'weight':'avg', 'dia':'min', 'height':'max'}).orderBy('max(height)', ascending=False).show(10)

+--------------+-----------------+--------+-----------+
|          team|      avg(weight)|min(dia)|max(height)|
+--------------+-----------------+--------+-----------+
|       Croatia|79.30434782608695|      02|        201|
|       Denmark| 82.6086956521739|      01|        200|
|     Argentina|75.56521739130434|      02|        199|
|       Belgium|79.56521739130434|      02|        199|
|       Iceland|80.73913043478261|      01|        198|
|        Sweden|78.82608695652173|      02|        198|
|       Nigeria|80.47826086956522|      01|        197|
|Korea Republic|74.43478260869566|      03|        197|
|        France|             80.0|      03|        197|
|        Panama|             80.0|      01|        197|
+--------------+-----------------+--------+-----------+
only showing top 10 rows

time: 1.03 s (started: 2023-09-19 16:50:40 +00:00)


In [50]:
df.groupBy('team').agg(avg('height'), min('height'), max('height')).orderBy('avg(height)', ascending=False).show(20)

+--------------+------------------+-----------+-----------+
|          team|       avg(height)|min(height)|max(height)|
+--------------+------------------+-----------+-----------+
|        Serbia|186.69565217391303|        169|        195|
|       Denmark| 186.6086956521739|        171|        200|
|       Germany| 185.7826086956522|        176|        195|
|        Sweden| 185.7391304347826|        177|        198|
|       Iceland|185.52173913043478|        170|        198|
|       Belgium|185.34782608695653|        169|        199|
|       Croatia| 185.2608695652174|        172|        201|
|       Nigeria|184.52173913043478|        172|        197|
|       IR Iran|184.47826086956522|        177|        194|
|        Russia| 184.3913043478261|        173|        196|
|       Senegal|183.65217391304347|        173|        196|
|        France|183.30434782608697|        168|        197|
|        Poland|183.17391304347825|        172|        195|
|       Tunisia|183.08695652173913|     

In [37]:
df.groupBy('team').agg(avg('weight')).orderBy('avg(weight)', ascending=True).show(10)

+--------------+-----------------+
|          team|      avg(weight)|
+--------------+-----------------+
|         Japan|71.52173913043478|
|  Saudi Arabia|73.04347826086956|
|      Portugal| 73.6086956521739|
|        Mexico|74.08695652173913|
|    Costa Rica| 74.1304347826087|
|Korea Republic|74.43478260869566|
|       Uruguay| 74.6086956521739|
|       Morocco|74.65217391304348|
|         Spain|74.73913043478261|
|       Tunisia|             75.0|
+--------------+-----------------+
only showing top 10 rows

time: 340 ms (started: 2023-09-19 16:50:41 +00:00)


# Instrução: PARTITION BY
Tem o conceito muito parecido com groupby, mas enquanto neste, as linhas iguais são agrupadas formando agrupamentos de instâncias de dados baseados em uma ou mais colunas.

*   row_number()
*   rank()
*   dense_rank()
*   persent_rank()
*   ntile()

**Obs.** A função orderBy() usada com Window.partitionBy() não é a mesma usada com as funções de agregação de groupBy(). Enquanto essa retorna um DataFrame o outro cria uma WindowSpec.

In [38]:
df.show(5)

+---------+---+---+------------------+----------+----------+--------------------+------+------+---+---+----+
|     team|  #|pos| fifa popular name|birth date|shirt name|                club|height|weight|dia|mes| ano|
+---------+---+---+------------------+----------+----------+--------------------+------+------+---+---+----+
|Argentina|  3| DF|TAGLIAFICO Nicolas|31.08.1992|TAGLIAFICO|      AFC Ajax (NED)|   169|    65| 31| 08|1992|
|Argentina| 22| MF|    PAVON Cristian|21.01.1996|     PAVÓN|CA Boca Juniors (...|   169|    65| 21| 01|1996|
|Argentina| 15| MF|    LANZINI Manuel|15.02.1993|   LANZINI|West Ham United F...|   167|    66| 15| 02|1993|
|Argentina| 18| DF|    SALVIO Eduardo|13.07.1990|    SALVIO|    SL Benfica (POR)|   167|    69| 13| 07|1990|
|Argentina| 10| FW|      MESSI Lionel|24.06.1987|     MESSI|  FC Barcelona (ESP)|   170|    72| 24| 06|1987|
+---------+---+---+------------------+----------+----------+--------------------+------+------+---+---+----+
only showing top 5 

row_number()

In [39]:
prt = Window.partitionBy('team').orderBy(desc('height'))
print(type(prt))
print(type(row_number()))
df.withColumn('row', row_number().over(prt)).show(10)

<class 'pyspark.sql.window.WindowSpec'>
<class 'pyspark.sql.column.Column'>
+---------+---+---+------------------+----------+----------+--------------------+------+------+---+---+----+---+
|     team|  #|pos| fifa popular name|birth date|shirt name|                club|height|weight|dia|mes| ano|row|
+---------+---+---+------------------+----------+----------+--------------------+------+------+---+---+----+---+
|Argentina|  6| DF|    FAZIO Federico|17.03.1987|     FAZIO|       AS Roma (ITA)|   199|    85| 17| 03|1987|  1|
|Argentina|  1| GK|     GUZMAN Nahuel|10.02.1986|    GUZMÁN|   Tigres UANL (MEX)|   192|    90| 10| 02|1986|  2|
|Argentina| 16| DF|       ROJO Marcos|20.03.1990|      ROJO|Manchester United...|   189|    82| 20| 03|1990|  3|
|Argentina| 12| GK|     ARMANI Franco|16.10.1986|    ARMANI|CA River Plate (ARG)|   189|    85| 16| 10|1986|  4|
|Argentina| 23| GK|CABALLERO Wilfredo|28.09.1981| CABALLERO|    Chelsea FC (ENG)|   186|    80| 28| 09|1981|  5|
|Argentina|  9| FW| 

In [40]:
# Selecionar os atletas mais altos de cada time.
prt = Window.partitionBy('team').orderBy(desc('height'))
df.withColumn('top', row_number().over(prt)).where("top = 1").show(10)

+----------+---+---+------------------+----------+-----------+--------------------+------+------+---+---+----+---+
|      team|  #|pos| fifa popular name|birth date| shirt name|                club|height|weight|dia|mes| ano|top|
+----------+---+---+------------------+----------+-----------+--------------------+------+------+---+---+----+---+
| Argentina|  6| DF|    FAZIO Federico|17.03.1987|      FAZIO|       AS Roma (ITA)|   199|    85| 17| 03|1987|  1|
| Australia| 12| GK|        JONES Brad|19.03.1982|      JONES|Feyenoord Rotterd...|   193|    87| 19| 03|1982|  1|
|   Belgium|  1| GK|  COURTOIS Thibaut|11.05.1992|   COURTOIS|    Chelsea FC (ENG)|   199|    91| 11| 05|1992|  1|
|    Brazil| 16| GK|            CASSIO|06.06.1987|     CASSIO|SC Corinthians (BRA)|   195|    92| 06| 06|1987|  1|
|  Colombia| 13| DF|        MINA Yerry|23.09.1994|    Y. MINA|  FC Barcelona (ESP)|   194|    95| 23| 09|1994|  1|
|Costa Rica| 19| DF|    WASTON Kendall|01.01.1988|  K. WASTON|Vancouver Whiteca.

rank(): Note como rank=3 se repete duas vezes e depois há um salto para rank=5. Esta é uma peculiaridade dessa função.

In [41]:
prt = Window.partitionBy('team').orderBy(desc('height'))
df.withColumn('rank', rank().over(prt)).show(10)

+---------+---+---+------------------+----------+----------+--------------------+------+------+---+---+----+----+
|     team|  #|pos| fifa popular name|birth date|shirt name|                club|height|weight|dia|mes| ano|rank|
+---------+---+---+------------------+----------+----------+--------------------+------+------+---+---+----+----+
|Argentina|  6| DF|    FAZIO Federico|17.03.1987|     FAZIO|       AS Roma (ITA)|   199|    85| 17| 03|1987|   1|
|Argentina|  1| GK|     GUZMAN Nahuel|10.02.1986|    GUZMÁN|   Tigres UANL (MEX)|   192|    90| 10| 02|1986|   2|
|Argentina| 16| DF|       ROJO Marcos|20.03.1990|      ROJO|Manchester United...|   189|    82| 20| 03|1990|   3|
|Argentina| 12| GK|     ARMANI Franco|16.10.1986|    ARMANI|CA River Plate (ARG)|   189|    85| 16| 10|1986|   3|
|Argentina| 23| GK|CABALLERO Wilfredo|28.09.1981| CABALLERO|    Chelsea FC (ENG)|   186|    80| 28| 09|1981|   5|
|Argentina|  9| FW|   HIGUAIN Gonzalo|10.12.1987|   HIGUAÍN|   Juventus FC (ITA)|   184|

dense_rank(): Aqui, mesmo que rank=3 se repita o próximo valor de rank é 4 e assim por diante. Não há saltos de valores.

In [42]:
prt = Window.partitionBy('team').orderBy(desc('height'))
df.withColumn('dense_rank', dense_rank().over(prt)).show(10)

+---------+---+---+------------------+----------+----------+--------------------+------+------+---+---+----+----------+
|     team|  #|pos| fifa popular name|birth date|shirt name|                club|height|weight|dia|mes| ano|dense_rank|
+---------+---+---+------------------+----------+----------+--------------------+------+------+---+---+----+----------+
|Argentina|  6| DF|    FAZIO Federico|17.03.1987|     FAZIO|       AS Roma (ITA)|   199|    85| 17| 03|1987|         1|
|Argentina|  1| GK|     GUZMAN Nahuel|10.02.1986|    GUZMÁN|   Tigres UANL (MEX)|   192|    90| 10| 02|1986|         2|
|Argentina| 16| DF|       ROJO Marcos|20.03.1990|      ROJO|Manchester United...|   189|    82| 20| 03|1990|         3|
|Argentina| 12| GK|     ARMANI Franco|16.10.1986|    ARMANI|CA River Plate (ARG)|   189|    85| 16| 10|1986|         3|
|Argentina| 23| GK|CABALLERO Wilfredo|28.09.1981| CABALLERO|    Chelsea FC (ENG)|   186|    80| 28| 09|1981|         4|
|Argentina|  9| FW|   HIGUAIN Gonzalo|10

persent_rank(): ranking relativo(percentual)

In [43]:
prt = Window.partitionBy('team').orderBy(desc('height'))
df.withColumn('persent_rank', percent_rank().over(prt)).show(10)

+---------+---+---+------------------+----------+----------+--------------------+------+------+---+---+----+--------------------+
|     team|  #|pos| fifa popular name|birth date|shirt name|                club|height|weight|dia|mes| ano|        persent_rank|
+---------+---+---+------------------+----------+----------+--------------------+------+------+---+---+----+--------------------+
|Argentina|  6| DF|    FAZIO Federico|17.03.1987|     FAZIO|       AS Roma (ITA)|   199|    85| 17| 03|1987|                 0.0|
|Argentina|  1| GK|     GUZMAN Nahuel|10.02.1986|    GUZMÁN|   Tigres UANL (MEX)|   192|    90| 10| 02|1986|0.045454545454545456|
|Argentina| 16| DF|       ROJO Marcos|20.03.1990|      ROJO|Manchester United...|   189|    82| 20| 03|1990| 0.09090909090909091|
|Argentina| 12| GK|     ARMANI Franco|16.10.1986|    ARMANI|CA River Plate (ARG)|   189|    85| 16| 10|1986| 0.09090909090909091|
|Argentina| 23| GK|CABALLERO Wilfredo|28.09.1981| CABALLERO|    Chelsea FC (ENG)|   186|  

ntile(): Divide cada partição em uma quantidade n de quartiles. Cada quartile recebe um valor único. Caso uma partição não seja divisível por n o aloritmo ajustará a quantidade de instâncias de dados pertencentes aos últimos quartiles de modo que a partição tenha n quartiles. Por exemplo, na seleção da Argentina, quando n=5, o penúltimo quartil tem 4 instâncias para poder formar mais um, o último quartil que terá apenas uma instância.

In [44]:
prt = Window.partitionBy('team').orderBy(desc('height'))
df.withColumn('ntile', ntile(5).over(prt)).show(20)

+---------+---+---+------------------+----------+----------+--------------------+------+------+---+---+----+-----+
|     team|  #|pos| fifa popular name|birth date|shirt name|                club|height|weight|dia|mes| ano|ntile|
+---------+---+---+------------------+----------+----------+--------------------+------+------+---+---+----+-----+
|Argentina|  6| DF|    FAZIO Federico|17.03.1987|     FAZIO|       AS Roma (ITA)|   199|    85| 17| 03|1987|    1|
|Argentina|  1| GK|     GUZMAN Nahuel|10.02.1986|    GUZMÁN|   Tigres UANL (MEX)|   192|    90| 10| 02|1986|    1|
|Argentina| 16| DF|       ROJO Marcos|20.03.1990|      ROJO|Manchester United...|   189|    82| 20| 03|1990|    1|
|Argentina| 12| GK|     ARMANI Franco|16.10.1986|    ARMANI|CA River Plate (ARG)|   189|    85| 16| 10|1986|    1|
|Argentina| 23| GK|CABALLERO Wilfredo|28.09.1981| CABALLERO|    Chelsea FC (ENG)|   186|    80| 28| 09|1981|    1|
|Argentina|  9| FW|   HIGUAIN Gonzalo|10.12.1987|   HIGUAÍN|   Juventus FC (ITA)

Lag function: O mesmo tipo de lag usado em séries temporais.

In [45]:
prt = Window.partitionBy('team').orderBy(desc('height'))
df.withColumn('lag', lag('weight', offset=2).over(prt)).show(10)

+---------+---+---+------------------+----------+----------+--------------------+------+------+---+---+----+----+
|     team|  #|pos| fifa popular name|birth date|shirt name|                club|height|weight|dia|mes| ano| lag|
+---------+---+---+------------------+----------+----------+--------------------+------+------+---+---+----+----+
|Argentina|  6| DF|    FAZIO Federico|17.03.1987|     FAZIO|       AS Roma (ITA)|   199|    85| 17| 03|1987|null|
|Argentina|  1| GK|     GUZMAN Nahuel|10.02.1986|    GUZMÁN|   Tigres UANL (MEX)|   192|    90| 10| 02|1986|null|
|Argentina| 16| DF|       ROJO Marcos|20.03.1990|      ROJO|Manchester United...|   189|    82| 20| 03|1990|  85|
|Argentina| 12| GK|     ARMANI Franco|16.10.1986|    ARMANI|CA River Plate (ARG)|   189|    85| 16| 10|1986|  90|
|Argentina| 23| GK|CABALLERO Wilfredo|28.09.1981| CABALLERO|    Chelsea FC (ENG)|   186|    80| 28| 09|1981|  82|
|Argentina|  9| FW|   HIGUAIN Gonzalo|10.12.1987|   HIGUAÍN|   Juventus FC (ITA)|   184|

Forward function: O mesmo tipo de forward usado em séries temporais.

In [46]:
prt = Window.partitionBy('team').orderBy(desc('height'))
df.withColumn('lead', lead('weight', offset=1).over(prt)).show(10)

+---------+---+---+------------------+----------+----------+--------------------+------+------+---+---+----+----+
|     team|  #|pos| fifa popular name|birth date|shirt name|                club|height|weight|dia|mes| ano|lead|
+---------+---+---+------------------+----------+----------+--------------------+------+------+---+---+----+----+
|Argentina|  6| DF|    FAZIO Federico|17.03.1987|     FAZIO|       AS Roma (ITA)|   199|    85| 17| 03|1987|  90|
|Argentina|  1| GK|     GUZMAN Nahuel|10.02.1986|    GUZMÁN|   Tigres UANL (MEX)|   192|    90| 10| 02|1986|  82|
|Argentina| 16| DF|       ROJO Marcos|20.03.1990|      ROJO|Manchester United...|   189|    82| 20| 03|1990|  85|
|Argentina| 12| GK|     ARMANI Franco|16.10.1986|    ARMANI|CA River Plate (ARG)|   189|    85| 16| 10|1986|  80|
|Argentina| 23| GK|CABALLERO Wilfredo|28.09.1981| CABALLERO|    Chelsea FC (ENG)|   186|    80| 28| 09|1981|  75|
|Argentina|  9| FW|   HIGUAIN Gonzalo|10.12.1987|   HIGUAÍN|   Juventus FC (ITA)|   184|